In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
pd.set_option('display.float_format', lambda x: '%.4f' %x)

In [3]:
import seaborn as sns
sns.set_context('paper', font_scale=1.3)
sns.set_style('white')

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from time import time

In [6]:
import matplotlib.ticker as tk

In [7]:
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline

In [8]:
import math 
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [9]:
df = pd.read_pickle('./Dataset/May01_09_cleaned.pkl')
df_may22_31 = pd.read_pickle('./Dataset/May22_31_cleaned.pkl')
df_may31_12 = pd.read_pickle('./Dataset/May31_June12_cleaned.pkl')
df_june15_16 = pd.read_pickle('./Dataset/June15_16_cleaned.pkl')

In [10]:
df.shape

(53407746, 12)

In [11]:
df_may22_31.shape

(60037385, 12)

In [12]:
df = df.append(df_may22_31, ignore_index=True)

In [13]:
df = df.append(df_may31_12, ignore_index= True)

In [14]:
df = df.append(df_june15_16, ignore_index= True)

In [15]:
print('Number of riws and columns:', df.shape)
df.tail(100)

Number of riws and columns: (186774936, 12)


,Source,Destination,Length,DateTime,Second,Minute,Hour,Day,weekday,YYYY,MM,DD
186774836,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.393272,34,15,9,Weekday,1,2007,6,1
186774837,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.394572,34,15,9,Weekday,1,2007,6,1
186774838,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.395836,34,15,9,Weekday,1,2007,6,1
186774839,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.397101,34,15,9,Weekday,1,2007,6,1
186774840,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.398377,34,15,9,Weekday,1,2007,6,1
186774841,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.399642,34,15,9,Weekday,1,2007,6,1
186774842,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.400908,34,15,9,Weekday,1,2007,6,1
186774843,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.402139,34,15,9,Weekday,1,2007,6,1
186774844,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.403379,34,15,9,Weekday,1,2007,6,1
186774845,195.165.45.170,192.168.1.199,1500,2007-06-01 09:15:34.404610,34,15,9,Weekday,1,2007,6,1


In [17]:
df.to_hdf('Dataset/Concated_Data.h5',key='df',mode='w')

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [ ]:
df = df.dropna(subset=['Length'])

In [ ]:
df['Length'] = pd.to_numeric(df['Length'], errors='coerce')

In [ ]:
print('Number of riws and column after removing missing values:',df.shape)
print('The time series starts from: ',df.DateTime.min())
print('The time series ends on: ',df.DateTime.max())

#### There are several statistical tests that we can use toquantify whether our data look as though it was drawn from a Gaussian distibution. And we will
####  use D'Agostino's K2 Test. In the SciPy implementation of the test, we will interpret the p value as follows. p <= alpha: reject H0, not normal. p > alpha: fail to reject H0, normal

In [ ]:
stat, p = stats.normaltest(df.Length)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha  = 0.05
if p > alpha:
    print('Data looks Gaussian (fail to reject Null hypothesis H0)')
else:
    print('Data does not look gaussian (reject H0)')

In [ ]:
sns.distplot(df.Length)
print('Kurtosis of normal distribution: {}'.format(stats.kurtosis(df.Length)))
print('Skewness of normal distribution: {}'.format(stats.skew(df.Length)))

### First Time Series plot

In [ ]:
df1 = df.loc[:, ['DateTime', 'Length']]
df1.set_index('DateTime', inplace = True)
df1.plot(figsize=(20,5))
plt.ylabel('Packet Length')
plt.legend().set_visible(False)
plt.tight_layout()
plt.title('Packet Header Length')
sns.despine(top=True)
plt.show()

### Box Plot of Daily vs Hourly packet length 

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.subplots_adjust(wspace=0.2)
sns.boxplot(x='Day', y='Length', data=df)
plt.xlabel('Day')
plt.title('Box Plot of daily packet length')
sns.despine(left=True)
plt.tight_layout()


plt.subplot(1,2,2)
sns.boxplot(x='Hour', y='Length', data=df)
plt.xlabel('Hour')
plt.title('Box plot of hourly packet length')
sns.despine(left=True)

In [ ]:
df.head()

### Global Active Power Distribution 

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
df['Length'].hist(bins=50)
plt.title('Packet length distribution')

plt.subplot(1,2,2)
stats.probplot(df['Length'], plot=plt)
df1.describe().T

### Average Packet Length Resampled Over Day, Hour

In [ ]:
fig = plt.figure(figsize=(18,16))
fig.subplots_adjust(hspace=1)
ax1 = fig.add_subplot(2,1,1)
ax1.plot(df1['Length'].resample('D').mean(), linewidth=1)
ax1.set_title('Mean Packet Length resampled over day')
ax1.tick_params(axis='both', which='major')


ax2 = fig.add_subplot(2,1,2)
ax2.plot(df1['Length'].resample('H').mean(), linewidth=1)
ax2.set_title('Mean Packet Length resampled over hour')
ax2.tick_params(axis='both', which='major')




### Plot Agg Packet Length Grouped by Day, Hour

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(2,2,1)
df.groupby('Day').Length.agg('mean').plot()
plt.xlabel('')
plt.title('Mean oPacket length by day')

plt.subplot(2,2,2)
df.groupby('Day').Length.agg('sum').plot()
plt.xlabel('')
plt.title('Sum Packet length by day')


plt.subplot(2,2,3)
df.groupby('Hour').Length.agg('mean').plot()
plt.xlabel('')
plt.title('Mean oPacket length by hour')

plt.subplot(2,2,4)
df.groupby('Hour').Length.agg('sum').plot()
plt.xlabel('')
plt.title('Sum Packet length by hour')

In [ ]:
pd.pivot_table(df, values='Length', columns='Day', index='Hour').plot(subplots = True, figsize=(12,12),  sharey=True)

In [ ]:
pd.pivot_table(df, values = 'Length', columns='Day', index='Hour')

## Stationary:
### In statistics, the Dickey–Fuller test tests the null hypothesis that a unit root is present in an autoregressive model. The alternative hypothesis is different depending on which version of the test is used, but is usually stationarity or trend-stationarity.
### Stationary series has constant mean and variance over time. Rolling average and the rolling standard deviation of time series do not change over time.

## Dickey-Fuller test
#### Null Hypothesis (H0): It suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.
#### Alternate Hypothesis (H1): It suggests the time series does not have a unit root, meaning it is stationary. It does not have time-dependent structure.
##### p-value > 0.05: Accept the null hypothesis (H0), the data has a unit root and is non-stationary.
##### p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [ ]:
df2 = df1.resample('H', how=np.sum)

In [ ]:
df2.head()

In [ ]:
def test_stationarity(timeseries):
    rolmean = timeseries.rolling(window=30).mean()
    rolstd = timeseries.rolling(window=30).std()
    
    
    plt.figure(figsize=(14,5))
    sns.despine(left=True)
    orig =plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label='Rolling Std')
    
    
    plt.legend(loc='best'); plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    
    print('<Results of Dickey-Fuller test>')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistics', 'p-value', '#Lags Used', ' Number of Observations Used'])
    
    
    for key, value in dftest[4].items():
        dfoutput['Critical value (%s)' %key] = value
    print(dfoutput)
    
test_stationarity(df2.Length)

## LSTM

### The following data pre-processing and feature engineering need to be done before construct the LSTM model.
##### Create the dataset, ensure all data is float.
##### Normalize the features.
##### Split into training and test sets.
##### Convert an array of values into a dataset matrix.
##### Reshape into X=t and Y=t+1.
##### Reshape input to be 3D (num_samples, num_timesteps, num_features).

In [ ]:
df_sum_secs = pd.DataFrame(df.groupby(['YYYY','MM','DD','Hour','Minute','Second']).agg({'Length': 'sum'})).reset_index()
df_sum_mins = pd.DataFrame(df.groupby(['YYYY','MM','DD','Hour','Minute']).agg({'Length': 'sum'})).reset_index()
df_sum_hrs = pd.DataFrame(df.groupby(['YYYY','MM','DD','Hour']).agg({'Length': 'sum'})).reset_index()

In [ ]:
df_sum_secs.head(10)

In [ ]:
df_sum_secs.shape

In [ ]:
dataset = df_sum_secs.Length.values #numpy.ndarray
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1,1))

In [ ]:
dataset.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)

In [ ]:
dataset

In [ ]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
def create_dataset(dataset, look_back=1):
    X,Y = [] , []
    
    for i in range(len(dataset) - look_back -1):
        a = dataset[i: (i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

## Model Architecture
#### LSTM with 100 neurons
#### Dropout 20%
### Use the MSE loss function
### 20 trainng epochs a batch size of 70

In [ ]:
keras.backend.clear_session()

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, Y_train, epochs=100, batch_size=70, validation_data=(X_test, Y_test),
                   callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

model.summary()

### Prediction

In [ ]:
 train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
# invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])


print('Train Mean Absolute Error:', mean_absolute_error(Y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test[0], test_predict[:,0])))

### Plot model loss

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

### Compare Actual vs. Prediction

In [ ]:
aa=[x for x in range(1000)]
plt.figure(figsize=(16,4))
plt.plot(aa, Y_test[0][1500:2500], marker='.', label="actual")
plt.plot(aa, test_predict[:,0][1500:2500], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Total Packer Length', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();
plt.savefig('seconds_2st_model.png')